In [1]:
import pandas as pd
import requests

In [2]:
data_url = 'https://publicapi.traffy.in.th/dump-csv-chadchart/bangkok_traffy.csv'
df = pd.read_csv(data_url)
df.head()

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-9LHDM6,{},NaN,ไม่มีภาพ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.48661,13.79386",1867 จรัญสนิทวงศ์ แขวง บางพลัด เขตบางพลัด กรุง...,บางพลัด,บางพลัด,กรุงเทพมหานคร,2021-09-01 10:44:55.353209+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:59:58.622268+00
1,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,NaN,2022-06-04 15:34:14.609206+00
2,2021-8GKAR9,{สายไฟ},ยังไม่มีหน่วยงานรับผิดชอบ,1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.57685,13.79704",335/31 ลาดพร้าว แขวง สามเสนนอก เขตห้วยขวาง กรุ...,สามเสนนอก,ห้วยขวาง,จังหวัดกรุงเทพมหานคร,2021-09-19 06:47:50.488685+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:30:07.293416+00
3,2021-AFPUXZ,"{ถนน,สะพาน}",NaN,1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52916,13.72338",37 10 ซอย สีลม 9 แขวง สีลม เขตบางรัก กรุงเทพมห...,สีลม,บางรัก,กรุงเทพมหานคร,2021-09-19 07:40:30.000781+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:30:04.507406+00
4,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,NaN,2022-06-21 08:21:09.532782+00


In [3]:
df.dtypes

ticket_id         object
type              object
organization      object
comment           object
photo             object
photo_after       object
coords            object
address           object
subdistrict       object
district          object
province          object
timestamp         object
state             object
star             float64
count_reopen     float64
last_activity     object
dtype: object

In [4]:
df.columns

Index(['ticket_id', 'type', 'organization', 'comment', 'photo', 'photo_after',
       'coords', 'address', 'subdistrict', 'district', 'province', 'timestamp',
       'state', 'star', 'count_reopen', 'last_activity'],
      dtype='object')

In [16]:
df_filter = df[['coords', 'photo', 'timestamp', 'type']]
df_filter = df_filter[df_filter.type == '{จราจร}']
df_filter = df_filter.dropna()
len(df_filter)

5139

In [6]:
photo_path = '../resources/photos/'
photos_filename = []
for index, row in df_filter.iterrows():
    img_url = row['photo']
    img_filename = f'{photo_path}/image_{index}.jpg'
    try: 
        response = requests.get(img_url)
    except:
        print(img_url)
        print(index)
        continue
    if response.status_code != 200:
        continue
    with open(img_filename, 'wb') as destination_image:
        destination_image.write(response.content)
    photos_filename.append(f'image_{index}.jpg')    


    

In [24]:
df_filename = pd.DataFrame({'image_name': photos_filename})
df_filter_no_index = df_filter.reset_index()
df_final = df_filter_no_index.join(df_filename)
df_final.head()


,index,coords,photo,timestamp,type,image_name
0,31,"100.51974,13.74059",https://storage.googleapis.com/traffy_public_b...,2021-12-10 05:01:04.955289+00,{จราจร},image_31.jpg
1,109,"100.55281,13.74219",https://storage.googleapis.com/traffy_public_b...,2021-12-18 07:36:06.837338+00,{จราจร},image_109.jpg
2,139,"100.49413,13.78444",https://storage.googleapis.com/traffy_public_b...,2021-12-22 01:24:57.223484+00,{จราจร},image_139.jpg
3,250,"100.56510,13.75622",https://storage.googleapis.com/traffy_public_b...,2022-01-23 05:02:21.192696+00,{จราจร},image_250.jpg
4,757,"100.47124,13.69833",https://storage.googleapis.com/traffy_public_b...,2022-05-26 00:41:04.314959+00,{จราจร},image_757.jpg


In [25]:
df_final = df_final.drop(['photo','type','index'], axis=1)
df_final.head()

,coords,timestamp,image_name
0,"100.51974,13.74059",2021-12-10 05:01:04.955289+00,image_31.jpg
1,"100.55281,13.74219",2021-12-18 07:36:06.837338+00,image_109.jpg
2,"100.49413,13.78444",2021-12-22 01:24:57.223484+00,image_139.jpg
3,"100.56510,13.75622",2022-01-23 05:02:21.192696+00,image_250.jpg
4,"100.47124,13.69833",2022-05-26 00:41:04.314959+00,image_757.jpg


In [27]:
df_final.to_csv('../resources/processed.csv', header=True)